# Wrappers and Extensions

MolPy uses both **composition** (Wrappers) and **inheritance** to extend functionality.

1.  **Wrappers**: Wrap an object to add features without modifying it (Composition).
2.  **Specialized Classes**: Subclasses like `Monomer` that add specific domain logic (Inheritance).

---


## 1. The Wrapper Class

The `Wrapper` class allows you to wrap any `Struct` (like `Atomistic`) and add new properties while keeping the original object intact.


In [ ]:
import molpy as mp
from molpy.core.wrappers.base import Wrapper
from molpy.core.atomistic import Atomistic


# Define a custom wrapper
class LabeledWrapper(Wrapper):
    def __init__(self, inner, label):
        super().__init__(inner)
        self.label = label

    def __repr__(self):
        return f"<Labeled: {self.label} | {self.inner}>"


# Create a molecule
mol = Atomistic()
mol.def_atom(symbol="O", xyz=[0, 0, 0])

# Wrap it
wrapped = LabeledWrapper(mol, label="WaterCenter")
print(wrapped)
print(f"Inner atoms: {wrapped.inner.n_atoms}")

## 2. Monomers

`Monomer` is a specialized class (inheriting from `Atomistic`) that adds **Ports**. Ports define connection points for building polymers.

It behaves like an `Atomistic` structure but has extra methods for managing ports.


In [ ]:
from molpy.core.wrappers.monomer import Monomer

# Create a Monomer
monomer = Monomer()
c1 = monomer.def_atom(symbol="C", xyz=[0, 0, 0])
c2 = monomer.def_atom(symbol="C", xyz=[1.5, 0, 0])
monomer.def_bond(c1, c2)

# Define ports (connection points)
monomer.define_port(name="head", target=c1)
monomer.define_port(name="tail", target=c2)

print(f"Monomer ports: {monomer.port_names()}")
print(f"Head port target: {monomer.get_port('head').target.symbol}")

## 3. Why use Wrappers?

- **Non-Invasive**: You don't need to change the core `Atomistic` class to add features.
- **Composition**: You can chain wrappers (e.g., `LabeledWrapper(ChargedWrapper(mol))`).
- **State Separation**: Keep your custom data (labels, scores) separate from the physical molecule data.
